<a href="https://colab.research.google.com/github/ParnaviSen03/Sentiment_Analysis_Twitter_Reddit/blob/main/Gemini_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from google.colab import files

In [10]:
# Upload file manually
uploaded = files.upload()

Saving technology_processed_data.csv to technology_processed_data.csv


In [3]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.dropna(subset=['text', 'Cluster'], inplace=True)
    df['Cluster'] = df['Cluster'].map({0: 'Positive', 1: 'Neutral', 2: 'Negative'})
    return df

def preprocess_text(text, tokenizer, max_length=128):
    return tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt').to(device)

def predict_sentiment(model, tokenizer, text):
    prompt = f"Classify the sentiment of the following text as Positive, Neutral, or Negative (Only output the sentiment): {text}"
    inputs = preprocess_text(prompt, tokenizer)
    with torch.no_grad():
        # Use max_new_tokens instead of max_length to control generated sequence length
        outputs = model.generate(**inputs, max_new_tokens=20)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [4]:
# Load model and tokenizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [11]:
# Load dataset
data_path = '/content/technology_processed_data.csv'  # Replace with actual dataset path
df1 = load_data(data_path)

In [12]:
# Split dataset
train_texts1, test_texts1, train_labels1, test_labels1 = train_test_split(df1['text'], df1['Cluster'], test_size=0.1, random_state=42)

In [13]:
# Predict sentiments
df1.loc[df1.index[test_texts1.index], 'predictions'] = [predict_sentiment(model, tokenizer, text) for text in test_texts1]

In [14]:
df1.to_csv('tech_gemini_data.csv', index=False)

In [16]:
files.download('tech_gemini_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>